* https://www.jeremyjordan.me/nn-learning-rate/
* https://github.com/davidtvs/pytorch-lr-finder/blob/master/examples/lrfinder_mnist.ipynb

In [ ]:
pip install torch-lr-finder

## Libraries

In [ ]:
import sys
sys.path.append("../input/timmmaster/")

In [ ]:
import os
import cv2
import pandas as pd
import numpy as np
import random
import timm
import time
import matplotlib.pyplot as plt

import torch
from torch import nn
from torch import optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision.io import read_image
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts

import albumentations as A
from albumentations.pytorch import ToTensorV2

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from torch_lr_finder import LRFinder

## Config

In [ ]:
class CONFIG:
    seed = 0
    image_size = 384
    model_name = "swin_large_patch4_window12_384"
    head_units = 32
    num_epochs = 9
    patience = 2
    batch_size = 8
    learning_rate = 1e-7
    weight_decay = 1e-4
    dropout = 0.6
    T_0 = 3
    n_splits = 5
    run_folds = [0,1,2,3,4]

## Setting the seed

In [ ]:
def set_seed(seed):
    '''
    Sets the seed of the entire notebook for reproducibility.
    '''
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # when running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)
    
set_seed(CONFIG.seed)

## Datasets

In [ ]:
INPUT_DIR = "../input/petfinder-pawpularity-score/"
TRAIN_DIR = INPUT_DIR + "train/"
TEST_DIR = INPUT_DIR + "test/"
train_df = pd.read_csv(INPUT_DIR + "train.csv")
train_df.shape

In [ ]:
train_df.head()

## Remove duplicate images

In [ ]:
remove_train_ids = [
'13d215b4c71c3dc603cd13fc3ec80181',
'5ef7ba98fc97917aec56ded5d5c2b099',
'1feb99c2a4cac3f3c4f8a4510421d6f5',
'5a642ecc14e9c57a05b8e010414011f2',
'0422cd506773b78a6f19416c98952407',
'9b3267c1652691240d78b7b3d072baf3',
'1059231cf2948216fcc2ac6afb4f8db8',
'8ffde3ae7ab3726cff7ca28697687a42',
'78a02b3cb6ed38b2772215c0c0a7f78e',
'bf8501acaeeedc2a421bac3d9af58bb7',
'fe47539e989df047507eaa60a16bc3fd',
'dd042410dc7f02e648162d7764b50900',
'988b31dd48a1bc867dbc9e14d21b05f6',
'e359704524fa26d6a3dcd8bfeeaedd2e',
'6ae42b731c00756ddd291fa615c822a1',
'9a0238499efb15551f06ad583a6fa951',
'a9513f7f0c93e179b87c01be847b3e4c',
'38426ba3cbf5484555f2b5e9504a6b03',
'cd909abf8f425d7e646eebe4d3bf4769',
'9f5a457ce7e22eecd0992f4ea17b6107',
'b148cbea87c3dcc65a05b15f78910715',
'3877f2981e502fe1812af38d4f511fd2',
'94c823294d542af6e660423f0348bf31',
'2b737750362ef6b31068c4a4194909ed',
'01430d6ae02e79774b651175edd40842',
'72b33c9c368d86648b756143ab19baeb',
'dbc47155644aeb3edd1bd39dba9b6953',
'b49ad3aac4296376d7520445a27726de',
'54563ff51aa70ea8c6a9325c15f55399',
'87c6a8f85af93b84594a36f8ffd5d6b8']
len(remove_train_ids)

In [ ]:
train_df = train_df.loc[~train_df['Id'].isin(remove_train_ids),:].reset_index(drop=True)
train_df.shape

## Create folds

In [ ]:
def create_folds(df, n_splits, seed):
    '''
    Assigns fold index to each image in df according to K-fold CV.
    '''
    df['fold'] = -1
    num_bins = int(np.floor(1 + np.log2(len(df))))
    df['bin'] = pd.cut(df['Pawpularity'], bins=num_bins, labels=False)
    
    skf = StratifiedKFold(n_splits = n_splits, shuffle=True, random_state=seed)
    for fold_idx, (trn_idx, val_idx) in enumerate(skf.split(X=df, y=df['bin'])):
        df.loc[val_idx,'fold'] = fold_idx
    
    df = df.drop('bin', axis=1)
    return df

In [ ]:
train_df = create_folds(train_df, CONFIG.n_splits, CONFIG.seed)
train_df.head()

## Dataset class

In [ ]:
class PetfinderDataset(Dataset):
    def __init__(self, train_df, img_dir, transform=None, target_transform=None):
        self.train_df = train_df
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform
        
    def __len__(self):
        return(len(self.train_df))
    
    def __getitem__(self, idx):
        img_path = self.img_dir + self.train_df.loc[idx,'Id'] + ".jpg"
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        label = self.train_df.loc[idx,'Pawpularity']
        if self.transform is not None:
            image = self.transform(image=image)["image"]
        if self.target_transform is not None:
            label = self.target_transform(label)
        return image, label

## Transforms

In [ ]:
data_transforms = {
    "train": A.Compose([
        A.Resize(CONFIG.image_size, CONFIG.image_size),
        A.HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.2),
        A.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
        ToTensorV2()], p=1.),
    
    "valid": A.Compose([
        A.Resize(CONFIG.image_size, CONFIG.image_size),
        A.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
        ToTensorV2()], p=1.)
}

target_transform = lambda x: x/100

## Model

In [ ]:
# check whether GPU is available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

In [ ]:
class PetfinderModel(nn.Module):
    def __init__(self, pretrained=False):
        super(PetfinderModel, self).__init__()
        self.model = timm.create_model(CONFIG.model_name, pretrained=pretrained, in_chans=3)
        self.model.head = nn.Linear(self.model.head.in_features, out_features=CONFIG.head_units)
        self.dropout = nn.Dropout(CONFIG.dropout)
        self.fc = nn.Linear(CONFIG.head_units, 1)
        self.out = nn.Sigmoid()
        
    def forward(self, image):
        x = self.model(image)
        x = self.dropout(x)
        x = self.fc(x)
        x = self.out(x)
        return x

## Loss and score functions

In [ ]:
def loss_fn(yhat, y):
    criterion = nn.BCELoss(reduction='mean')
    return criterion(yhat.view(-1).float(), y.view(-1).float())

## Create data loaders for CV

In [ ]:
def create_loaders_one_fold(df: pd.DataFrame, img_dir: str, fold: int):
    '''
    Creates training and validation data loaders for one fold in K-fold CV.
    '''
    df_train = df[df['fold'] != fold].reset_index(drop=True)
    df_tst = df[df['fold'] == fold].reset_index(drop=True)
    
    # validation set for ES
    num_bins = int(np.floor(1 + np.log2(len(df_train))))
    df_train['bin'] = pd.cut(df_train['Pawpularity'], bins=num_bins, labels=False)
    df_trn, df_val = train_test_split(df_train, test_size=0.1, stratify=df_train['bin'])
    df_trn = df_trn.drop('bin', axis=1).reset_index(drop=True)
    df_val = df_val.drop('bin', axis=1).reset_index(drop=True)
    
    # create Dataset objects
    trn_dataset = PetfinderDataset(df_trn, img_dir, transform=data_transforms['train'], target_transform=target_transform)
    val_dataset = PetfinderDataset(df_val, img_dir, transform=data_transforms['valid'], target_transform=target_transform)
    tst_dataset = PetfinderDataset(df_tst, img_dir, transform=data_transforms['valid'], target_transform=target_transform)
    
    # create Dataloader objects
    trn_loader = DataLoader(trn_dataset, CONFIG.batch_size, num_workers=2, shuffle=True, pin_memory=True, drop_last=True)
    val_loader = DataLoader(val_dataset, CONFIG.batch_size*2, num_workers=2, shuffle=False, pin_memory=True, drop_last=False)
    tst_loader = DataLoader(tst_dataset, CONFIG.batch_size*2, num_workers=2, shuffle=False, pin_memory=True, drop_last=False)
    
    return trn_loader, val_loader, tst_loader

## LR finder

In [ ]:
def lr_finder_one_fold(fold:int):
    # create loaders
    trn_loader, val_loader, tst_loader = create_loaders_one_fold(train_df, TRAIN_DIR, fold)
    # create model
    model = PetfinderModel(pretrained=True)
    # move the model to the GPU
    model.to(device)
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=CONFIG.learning_rate, weight_decay=1e-2)
    lr_finder = LRFinder(model, optimizer, loss_fn, device=device)
    lr_finder.range_test(trn_loader, end_lr=1e-1, num_iter=100)
    lr_finder.plot()
    lr_finder.reset()

In [ ]:
lr_finder_one_fold(0)